# Direct Coords Approach

Instead of a IntervalIndex, we can encode metadata as coordinates that share the time dimension.

## Setup

In [1]:
import numpy as np
import xarray as xr

## Creating the Dataset

We create a dataset where `word` is a coordinate on the `time` dimension.

In [2]:
# Create sample data
T = 1000
C = 2
times = np.linspace(0, 120, T)
data = np.random.rand(C, T)

# Define word boundaries
breaks = np.array([0, 333, 666, 1000])

# Create word labels for each time point
words = np.array(["red"] * T)
words[breaks[0] : breaks[1]] = "red"
words[breaks[1] : breaks[2]] = "green"
words[breaks[2] :] = "blue"

# Create Dataset with word as a coord on the time dimension
ds = xr.Dataset(
    {"data": (("C", "time"), data)}, coords={"time": times, "word": ("time", words)}
).set_xindex("word")
ds

<xarray.Dataset> Size: 36kB
Dimensions:  (C: 2, time: 1000)
Coordinates:
  * time     (time) float64 8kB 0.0 0.1201 0.2402 0.3604 ... 119.8 119.9 120.0
  * word     (time) <U3 12kB 'red' 'red' 'red' 'red' ... 'blu' 'blu' 'blu' 'blu'
Dimensions without coordinates: C
Data variables:
    data     (C, time) float64 16kB 0.4334 0.3878 0.8662 ... 0.7358 0.7474

## What Works

### Time slicing:

In [3]:
ds.sel(time=slice(0.15, 15.5))

<xarray.Dataset> Size: 5kB
Dimensions:  (C: 2, time: 128)
Coordinates:
  * time     (time) float64 1kB 0.2402 0.3604 0.4805 0.6006 ... 15.26 15.38 15.5
  * word     (time) <U3 2kB 'red' 'red' 'red' 'red' ... 'red' 'red' 'red' 'red'
Dimensions without coordinates: C
Data variables:
    data     (C, time) float64 2kB 0.8662 0.08782 0.6194 ... 0.9323 0.07574

### Selection by word label works:

In [4]:
ds.sel(word="red")

<xarray.Dataset> Size: 12kB
Dimensions:  (C: 2, time: 333)
Coordinates:
  * time     (time) float64 3kB 0.0 0.1201 0.2402 0.3604 ... 39.64 39.76 39.88
  * word     (time) <U3 4kB 'red' 'red' 'red' 'red' ... 'red' 'red' 'red' 'red'
Dimensions without coordinates: C
Data variables:
    data     (C, time) float64 5kB 0.4334 0.3878 0.8662 ... 0.2404 0.2704 0.2363

## Limitations

### 1. Annoying to construct

The natural representation of metadata is often: `onset, duration, word`. To create the dense array we need to manually expand this into a value for every time point.

### 2. No `isel` by word

Since `word` is on the `time` dimension, there's no `word` dimension to index into:

In [5]:
# This doesn't work - word is not a dimension
try:
    ds.isel(word=0)
except ValueError as e:
    print(f"Error: {e}")

Error: Dimensions {'word'} do not exist. Expected one or more of FrozenMappingWarningOnValuesAccess({'C': 2, 'time': 1000})


### 3. Interval info is obscured

Important questions become hard to answer:
- What was the total duration of the 3rd word?
- What are the exact interval boundaries?

### 4. Constrained to measurement time points

If metadata events happen at times not in your measurement grid, you lose that precision. For example, if you sample monthly but an event happened mid-month, you can't represent that exactly.

### 5. Can't drop coord as index when it becomes scalar

When a selection reduces the coordinate to a single value, you can't easily drop it from being an index:

In [6]:
# Selecting a single word returns all matching time points
result = ds.sel(word="red")
print(f"Result has {len(result.time)} time points")
print(f"word coord: {result.word.values}")

# But word is still an index on the time dimension
# If you want to drop it, you can't easily do so when it's scalar
# result.drop_indexes('word')  # This can cause issues

Result has 333 time points
word coord: ['red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red' 'red'
 'red' 'red'

## Advantages

Despite the limitations, this approach:
- Clearly shows that word spans time
- Uses only standard xarray features
- Is simple to understand